#**VGG16/19 to tflite model with integer weights**

We will use tensorflow version 2.10 for our objective

In [1]:
!pip install --upgrade tensorflow==2.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 47.6 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2

Confirming version of tensorflow

In [1]:
import tensorflow as tf
print(tf.__version__)

2.10.0


**Loading the model**

In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions
import numpy as np

# Load the pre-trained VGG-16 model
model = VGG19(weights='imagenet', include_top=True)

574710816/574710816 [==============================] - 5s 0us/step


**Cloning this git repository containing images for validation**

In [3]:
!git clone https://github.com/ndb796/Small-ImageNet-Validation-Dataset-1000-Classes

Cloning into 'Small-ImageNet-Validation-Dataset-1000-Classes'...
remote: Enumerating objects: 6022, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6022 (delta 1), reused 0 (delta 0), pack-reused 6016
Receiving objects: 100% (6022/6022), 624.87 MiB | 40.12 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Updating files: 100% (5005/5005), done.


**Tflite conversion**

In [4]:
import os
import numpy as np
import random
from PIL import Image

# Define the path to the directory containing images
image_dir = '/content/Small-ImageNet-Validation-Dataset-1000-Classes/ILSVRC2012_img_val_subset'

selected_dirs = random.sample(os.listdir(image_dir), 5)  # Choose 5 random directories

# Defining the representative dataset function

def representative_dataset():
    # Iterate over each selected subdirectory
  for subdir in selected_dirs:
    subdir_path = os.path.join(image_dir, subdir)
    if os.path.isdir(subdir_path):
      # List all images in the subdirectory
      images = os.listdir(subdir_path)
      # Shuffle the list of images
      random.shuffle(images)
      # Process the first image in the shuffled list
      image_name = images[0]
      image_path = os.path.join(subdir_path, image_name)

      # Load and preprocess the image
      img = Image.open(image_path).resize((224, 224))
      # Skip grayscale images
      if len(np.array(img).shape) == 2:
        continue
      img_array = np.array(img)
      # img_array = img_array.astype(np.uint8)
      img_array = preprocess_input(img_array)  # Preprocess the input image
      img_array = np.expand_dims(img_array, axis=0)
      # Calculate statistics (assuming RGB image channels)
      min_val = np.amin(img_array, axis=(0, 1, 2))  # Min value for each channel
      max_val = np.amax(img_array, axis=(0, 1, 2))  # Max value for each channel
      stats = [min_val[0], min_val[1], min_val[2], max_val[0], max_val[1], max_val[2]]
      yield {
          model.inputs[0].name: np.array(img_array,dtype=np.float32),  # Replace 'model.inputs[0].name' with your actual input name
          'stats': np.array(stats)
      }



# Set up the TensorFlow Lite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.calibrate_inputs = True  # Enable calibration
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
# converter.experimental_new_quantizer = True  # Enable experimental quantization

# Convert the model to TensorFlow Lite format
tflite_quant_model = converter.convert()

# Save the quantized TensorFlow Lite model to a file
with open('vgg19_tfl.tflite', 'wb') as f:
    f.write(tflite_quant_model)


/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


**Uncomment this codeblock to check prediction of original model on an image**

In [ ]:
# from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
# import numpy as np

# # Load the image
# img_path = 'Enter image path'
# img = image.load_img(img_path, target_size=(224, 224))

# # Convert the image to a numpy array
# img_array = image.img_to_array(img)

# # Expand the dimensions to create a batch (VGG16 expects batches)
# img_batch = np.expand_dims(img_array, axis=0)

# # Preprocess the input (normalization, etc.)
# img_batch = preprocess_input(img_batch)

# # Make predictions
# predictions = model.predict(img_batch)
# print(np.argmax(predictions))
# # print(predictions)
# # Decode predictions (optional)
# decoded_predictions = decode_predictions(predictions, top=3)[0]

# # Print the top 3 predictions
# for pred in decoded_predictions:
#     print(pred)

**Prediction of tflite model on an image from validation set**

In [5]:
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing import image

# Load the image
img_path = "/content/Small-ImageNet-Validation-Dataset-1000-Classes/ILSVRC2012_img_val_subset/319/ILSVRC2012_val_00000418.JPEG"
img = image.load_img(img_path, target_size=(224, 224))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Expand the dimensions to create a batch (TFLite expects batches)
img_batch = np.expand_dims(img_array, axis=0)

# Preprocess the input
img_batch = preprocess_input(img_batch)

# Convert the input data to UINT8
img_batch = (np.round((img_batch/1.0774157047271729)-13)-128).astype(np.uint8)

# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='/content/vgg19_tfl.tflite')
interpreter.allocate_tensors()

# Set the input tensor for the TensorFlow Lite model
input_details = interpreter.get_input_details()
interpreter.set_tensor(input_details[0]['index'], img_batch)

# Run inference
interpreter.invoke()

# Get the output tensor
output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])

predicted_class_index = np.argmax(output_data)
print("Predicted class index:", predicted_class_index)

Predicted class index: 319


**Data Generator for running inference on large number of images**

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess the ImageNet validation dataset
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    '/content/Small-ImageNet-Validation-Dataset-1000-Classes/ILSVRC2012_img_val_subset',
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical',
    shuffle=True,
    classes=[str(i) for i in range(0, 1000)]  # Use category names as classes
)

# Print class indices
print("Class Indices:", validation_generator.class_indices)

# Print the number of images
num_images = validation_generator.samples
print("Number of images:", num_images)



Found 5000 images belonging to 1000 classes.
Class Indices: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '50': 50, '51': 51, '52': 52, '53': 53, '54': 54, '55': 55, '56': 56, '57': 57, '58': 58, '59': 59, '60': 60, '61': 61, '62': 62, '63': 63, '64': 64, '65': 65, '66': 66, '67': 67, '68': 68, '69': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '80': 80, '81': 81, '82': 82, '83': 83, '84': 84, '85': 85, '86': 86, '87': 87, '88': 88, '89': 89, '90': 90, '91': 91, '92': 92, '93': 93, '94': 94, '95': 95,

**Comparing original model vs tflite's accuracy**

In [8]:
# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='/content/vgg19_tfl.tflite')
interpreter.allocate_tensors()

# Initialize variables to store correct predictions
vgg_correct = 0
tflite_correct = 0
total_samples = 0
desired_samples = 100 # Number of images from validation set you want to evaluate

# Iterate over the validation dataset and make predictions
for x_batch, y_batch in validation_generator:
    batch_size = len(x_batch)
    total_samples += batch_size

    for i in range(batch_size):
        # VGG16 predictions
        vgg_prediction = np.argmax(model.predict(np.expand_dims(x_batch[i], axis=0)))
        vgg_correct += int(vgg_prediction == np.argmax(y_batch[i]))

        # TensorFlow Lite model predictions
        # Preprocess input image (same as for original model)
        img_batch = np.expand_dims(x_batch[i], axis=0)
        # x_batch_preprocessed = preprocess_input(img_batch)
        img_batch = (np.round((img_batch/1.0774157047271729)-13)-128).astype(np.uint8)

        # Set the input tensor for the TensorFlow Lite model
        input_details = interpreter.get_input_details()
        interpreter.set_tensor(input_details[0]['index'], img_batch)

        # Run inference
        interpreter.invoke()

        # Get the output tensor
        output_details = interpreter.get_output_details()

        # Get output tensor from TensorFlow Lite model
        tflite_prediction = np.argmax(interpreter.get_tensor(output_details[0]['index']))
        tflite_correct += int(tflite_prediction == np.argmax(y_batch[i]))
        # print(f'model: {tflite_prediction} actual: {np.argmax(y_batch[i])}')
    if total_samples >= desired_samples:
        break

# Calculate accuracies
vgg_accuracy = vgg_correct / total_samples
tflite_accuracy = tflite_correct / total_samples

print("VGG19 Model's Accuracy:", vgg_accuracy)
print("TFLite Model's Accuracy:", tflite_accuracy)

1/1 [==============================] - 1s 1s/step
VGG19 Model's Accuracy: 0.69
TFLite Model's Accuracy: 0.62


**Conclusion: We have successfully converted the model to its tflte integer version without much loss in accuracy**